In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline
from sklearn.externals import joblib

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
df = pd.read_pickle("../dataset.pkl")

In [3]:
x = df.iloc[:,:483]
x.columns
#483 = 3(adm_days,gender,age)+480(lab values)

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483)

In [4]:
y = df.iloc[:,483:]


In [5]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
#train,test,val 
seed = 40
x_tra,x_test,y_tra,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size = 0.2,random_state=seed)

In [7]:
y.shape

(58976, 20)

In [8]:
x_train.shape

(40103, 483)

In [9]:
y_train.shape

(40103, 20)

In [10]:
x_test.shape

(8847, 483)

In [11]:
y_test.shape

(8847, 20)

In [15]:
np.unique(y_train)

array([0, 1])

In [14]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
sample_weights = [1-3032/58976,1-8924/58976,1-35608/58976,1-19366/58976,
                 1-16271/58976,1-20008/58976,1-43243/58976,
                 1-25281/58976,1-21200/58976,1-21782/58976,1-166/58976,          
                 1-8316/58976,1-10053/58976,1-3205/58976,
                 1-5492/58976,1-19454/58976,1-1757/58976,1-23352/58976,1-16621/58976,1-34350/58976]
weight_rf = [{0:3032/58976,1:1-3032/58976},{0:8924/58976,1:1-8924/58976},{0:1-35608/58976,1:35608/58976},{0:19366/58976,1:1-19366/58976},
            {0:16271/58976,1:1-16271/58976},{0:20008/58976,1:1-20008/58976},{0:1-43243/58976,1:43243/58976},
            {0:25281/58976,1:1-25281/58976},{0:21200/58976,1:1-21200/58976},{0:21782/58976,1:1-21782/58976},{0:166/58976,1:1-166/58976},
            {0:8316/58976,1:1-8316/58976},{0:10053/58976,1:1-10053/58976},{0:3205/58976,1:1-3205/58976},
            {0:5492/58976,1:1-5492/58976},{0:19454/58976,1:1-19454/58976},{0:1757/58976,1:1-1757/58976},
            {0:23352/58976,1:1-23352/58976},{0:16621/58976,1:1-16621/58976},{0:1-34350/58976,1:34350/58976}]*58976
weights = [0.286,0.714] #this for catboost
#[0,1] classes 0.286 is probabbility of 1 in dataset(icd) so i flipped it over as to give more weight to 1

#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
clf_multilabel = OneVsRestClassifier(CatBoostClassifier(1500,class_weights=weights,task_type="GPU",
                           devices='0:1'))
# what are u trying to assign 1500 to?try

In [34]:
'''parameters = {
    
    
    "learning_rate":[0.1, 0.2, 0.3, 0.4]
}

model_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,
                             scoring = 'roc_auc')

model_tunning.fit(x_train, y_train)

print(model_tunning.best_score_)
print(model_tunning.best_params_)'''

'parameters = {\n    \n    \n    "learning_rate":[0.1, 0.2, 0.3, 0.4]\n}\n\nmodel_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,\n                             scoring = \'roc_auc\')\n\nmodel_tunning.fit(x_train, y_train)\n\nprint(model_tunning.best_score_)\nprint(model_tunning.best_params_)'

In [15]:
#estimator.get_params().keys()

In [ ]:
cbc = clf_multilabel.fit(x_train,y_train)
joblib.dump(cbc, "CAT_40.pkl")

Learning rate set to 0.033184


0:	learn: 0.6642199	total: 205ms	remaining: 5m 6s
1:	learn: 0.6394182	total: 393ms	remaining: 4m 54s
2:	learn: 0.6155382	total: 568ms	remaining: 4m 43s
3:	learn: 0.5928931	total: 744ms	remaining: 4m 38s
4:	learn: 0.5717403	total: 921ms	remaining: 4m 35s
5:	learn: 0.5532848	total: 1.1s	remaining: 4m 33s
6:	learn: 0.5359195	total: 1.29s	remaining: 4m 34s
7:	learn: 0.5192389	total: 1.47s	remaining: 4m 33s
8:	learn: 0.5046545	total: 1.66s	remaining: 4m 34s
9:	learn: 0.4913094	total: 1.86s	remaining: 4m 36s
10:	learn: 0.4785410	total: 2.05s	remaining: 4m 37s
11:	learn: 0.4669202	total: 2.23s	remaining: 4m 36s
12:	learn: 0.4568028	total: 2.4s	remaining: 4m 35s
13:	learn: 0.4469857	total: 2.58s	remaining: 4m 33s
14:	learn: 0.4381398	total: 2.76s	remaining: 4m 32s
15:	learn: 0.4297936	total: 2.93s	remaining: 4m 31s
16:	learn: 0.4225560	total: 3.11s	remaining: 4m 31s
17:	learn: 0.4156098	total: 3.28s	remaining: 4m 30s
18:	learn: 0.4088222	total: 3.46s	remaining: 4m 29s
19:	learn: 0.4027583	tota

159:	learn: 0.2931228	total: 29.1s	remaining: 4m 4s
160:	learn: 0.2929396	total: 29.3s	remaining: 4m 3s
161:	learn: 0.2928759	total: 29.5s	remaining: 4m 3s
162:	learn: 0.2928144	total: 29.7s	remaining: 4m 3s
163:	learn: 0.2927713	total: 29.9s	remaining: 4m 3s
164:	learn: 0.2927244	total: 30.1s	remaining: 4m 3s
165:	learn: 0.2926752	total: 30.3s	remaining: 4m 3s
166:	learn: 0.2924615	total: 30.5s	remaining: 4m 3s
167:	learn: 0.2923901	total: 30.7s	remaining: 4m 3s
168:	learn: 0.2923141	total: 30.9s	remaining: 4m 3s
169:	learn: 0.2921619	total: 31.1s	remaining: 4m 3s
170:	learn: 0.2921171	total: 31.3s	remaining: 4m 3s
171:	learn: 0.2920782	total: 31.5s	remaining: 4m 2s
172:	learn: 0.2919840	total: 31.7s	remaining: 4m 2s
173:	learn: 0.2919559	total: 31.8s	remaining: 4m 2s
174:	learn: 0.2919383	total: 32s	remaining: 4m 2s
175:	learn: 0.2918693	total: 32.2s	remaining: 4m 2s
176:	learn: 0.2917981	total: 32.4s	remaining: 4m 2s
177:	learn: 0.2917703	total: 32.6s	remaining: 4m 2s
178:	learn: 0.

317:	learn: 0.2816051	total: 59.3s	remaining: 3m 40s
318:	learn: 0.2815888	total: 59.5s	remaining: 3m 40s
319:	learn: 0.2815195	total: 59.7s	remaining: 3m 40s
320:	learn: 0.2814636	total: 59.9s	remaining: 3m 39s
321:	learn: 0.2814418	total: 1m	remaining: 3m 39s
322:	learn: 0.2813708	total: 1m	remaining: 3m 39s
323:	learn: 0.2813577	total: 1m	remaining: 3m 39s
324:	learn: 0.2812644	total: 1m	remaining: 3m 39s
325:	learn: 0.2812233	total: 1m	remaining: 3m 39s
326:	learn: 0.2811900	total: 1m 1s	remaining: 3m 38s
327:	learn: 0.2810130	total: 1m 1s	remaining: 3m 38s
328:	learn: 0.2809333	total: 1m 1s	remaining: 3m 38s
329:	learn: 0.2808854	total: 1m 1s	remaining: 3m 38s
330:	learn: 0.2807898	total: 1m 1s	remaining: 3m 38s
331:	learn: 0.2807459	total: 1m 1s	remaining: 3m 38s
332:	learn: 0.2806718	total: 1m 2s	remaining: 3m 37s
333:	learn: 0.2806096	total: 1m 2s	remaining: 3m 37s
334:	learn: 0.2805893	total: 1m 2s	remaining: 3m 37s
335:	learn: 0.2804607	total: 1m 2s	remaining: 3m 37s
336:	lea

471:	learn: 0.2734574	total: 1m 28s	remaining: 3m 13s
472:	learn: 0.2734478	total: 1m 28s	remaining: 3m 13s
473:	learn: 0.2734344	total: 1m 29s	remaining: 3m 12s
474:	learn: 0.2734024	total: 1m 29s	remaining: 3m 12s
475:	learn: 0.2733800	total: 1m 29s	remaining: 3m 12s
476:	learn: 0.2732601	total: 1m 29s	remaining: 3m 12s
477:	learn: 0.2732077	total: 1m 29s	remaining: 3m 12s
478:	learn: 0.2731714	total: 1m 30s	remaining: 3m 11s
479:	learn: 0.2731166	total: 1m 30s	remaining: 3m 11s
480:	learn: 0.2730830	total: 1m 30s	remaining: 3m 11s
481:	learn: 0.2730273	total: 1m 30s	remaining: 3m 11s
482:	learn: 0.2729725	total: 1m 30s	remaining: 3m 11s
483:	learn: 0.2729615	total: 1m 31s	remaining: 3m 11s
484:	learn: 0.2729514	total: 1m 31s	remaining: 3m 10s
485:	learn: 0.2727839	total: 1m 31s	remaining: 3m 10s
486:	learn: 0.2727759	total: 1m 31s	remaining: 3m 10s
487:	learn: 0.2727679	total: 1m 31s	remaining: 3m 10s
488:	learn: 0.2726978	total: 1m 31s	remaining: 3m 10s
489:	learn: 0.2726740	total:

...

160:	learn: 0.4340024	total: 7.68s	remaining: 1m 3s
161:	learn: 0.4339290	total: 7.73s	remaining: 1m 3s
162:	learn: 0.4338362	total: 7.78s	remaining: 1m 3s
163:	learn: 0.4337376	total: 7.83s	remaining: 1m 3s
164:	learn: 0.4336305	total: 7.88s	remaining: 1m 3s
165:	learn: 0.4334941	total: 7.93s	remaining: 1m 3s
166:	learn: 0.4333563	total: 7.98s	remaining: 1m 3s
167:	learn: 0.4332501	total: 8.04s	remaining: 1m 3s
168:	learn: 0.4331531	total: 8.09s	remaining: 1m 3s
169:	learn: 0.4330805	total: 8.13s	remaining: 1m 3s
170:	learn: 0.4330364	total: 8.18s	remaining: 1m 3s
171:	learn: 0.4329620	total: 8.23s	remaining: 1m 3s
172:	learn: 0.4328890	total: 8.28s	remaining: 1m 3s
173:	learn: 0.4328138	total: 8.31s	remaining: 1m 3s
174:	learn: 0.4327461	total: 8.36s	remaining: 1m 3s
175:	learn: 0.4326684	total: 8.41s	remaining: 1m 3s
176:	learn: 0.4326045	total: 8.46s	remaining: 1m 3s
177:	learn: 0.4325176	total: 8.5s	remaining: 1m 3s
178:	learn: 0.4324425	total: 8.55s	remaining: 1m 3s
179:	learn: 0

324:	learn: 0.4237410	total: 15.5s	remaining: 55.9s
325:	learn: 0.4236934	total: 15.5s	remaining: 55.8s
326:	learn: 0.4235968	total: 15.6s	remaining: 55.8s
327:	learn: 0.4235719	total: 15.6s	remaining: 55.7s
328:	learn: 0.4235463	total: 15.6s	remaining: 55.7s
329:	learn: 0.4234591	total: 15.7s	remaining: 55.6s
330:	learn: 0.4234030	total: 15.7s	remaining: 55.6s
331:	learn: 0.4233553	total: 15.8s	remaining: 55.5s
332:	learn: 0.4232522	total: 15.8s	remaining: 55.5s
333:	learn: 0.4231709	total: 15.9s	remaining: 55.5s
334:	learn: 0.4231182	total: 15.9s	remaining: 55.4s
335:	learn: 0.4230463	total: 16s	remaining: 55.4s
336:	learn: 0.4230033	total: 16s	remaining: 55.3s
337:	learn: 0.4229736	total: 16.1s	remaining: 55.3s
338:	learn: 0.4228878	total: 16.1s	remaining: 55.2s
339:	learn: 0.4228256	total: 16.2s	remaining: 55.2s
340:	learn: 0.4227849	total: 16.2s	remaining: 55.1s
341:	learn: 0.4227426	total: 16.3s	remaining: 55.1s
342:	learn: 0.4226968	total: 16.3s	remaining: 55s
343:	learn: 0.4226

486:	learn: 0.4151322	total: 23.3s	remaining: 48.5s
487:	learn: 0.4151088	total: 23.3s	remaining: 48.4s
488:	learn: 0.4150842	total: 23.4s	remaining: 48.4s
489:	learn: 0.4150498	total: 23.4s	remaining: 48.3s
490:	learn: 0.4150240	total: 23.5s	remaining: 48.3s
491:	learn: 0.4150067	total: 23.5s	remaining: 48.2s
492:	learn: 0.4149727	total: 23.6s	remaining: 48.2s
493:	learn: 0.4148840	total: 23.6s	remaining: 48.1s
494:	learn: 0.4148162	total: 23.7s	remaining: 48.1s
495:	learn: 0.4147707	total: 23.7s	remaining: 48s
496:	learn: 0.4147583	total: 23.8s	remaining: 48s
497:	learn: 0.4147323	total: 23.8s	remaining: 47.9s
498:	learn: 0.4147053	total: 23.9s	remaining: 47.9s
499:	learn: 0.4146664	total: 23.9s	remaining: 47.8s
500:	learn: 0.4146228	total: 24s	remaining: 47.8s
501:	learn: 0.4145907	total: 24s	remaining: 47.7s
502:	learn: 0.4145437	total: 24.1s	remaining: 47.7s
503:	learn: 0.4144971	total: 24.1s	remaining: 47.7s
504:	learn: 0.4144735	total: 24.2s	remaining: 47.6s
505:	learn: 0.414440

646:	learn: 0.4092592	total: 30.9s	remaining: 40.7s
647:	learn: 0.4092288	total: 31s	remaining: 40.7s
648:	learn: 0.4092017	total: 31s	remaining: 40.6s
649:	learn: 0.4091926	total: 31s	remaining: 40.6s
650:	learn: 0.4091739	total: 31.1s	remaining: 40.5s
651:	learn: 0.4091182	total: 31.1s	remaining: 40.5s
652:	learn: 0.4090763	total: 31.2s	remaining: 40.4s
653:	learn: 0.4090479	total: 31.2s	remaining: 40.4s
654:	learn: 0.4090232	total: 31.3s	remaining: 40.4s
655:	learn: 0.4089842	total: 31.3s	remaining: 40.3s
656:	learn: 0.4089696	total: 31.4s	remaining: 40.3s
657:	learn: 0.4089425	total: 31.4s	remaining: 40.2s
658:	learn: 0.4089306	total: 31.5s	remaining: 40.2s
659:	learn: 0.4089152	total: 31.5s	remaining: 40.1s
660:	learn: 0.4089017	total: 31.6s	remaining: 40.1s
661:	learn: 0.4088879	total: 31.6s	remaining: 40s
662:	learn: 0.4088661	total: 31.7s	remaining: 40s
663:	learn: 0.4088396	total: 31.7s	remaining: 39.9s
664:	learn: 0.4088026	total: 31.8s	remaining: 39.9s
665:	learn: 0.4087698	

806:	learn: 0.4045787	total: 38.5s	remaining: 33s
807:	learn: 0.4045455	total: 38.5s	remaining: 33s
808:	learn: 0.4045078	total: 38.6s	remaining: 32.9s
809:	learn: 0.4044884	total: 38.6s	remaining: 32.9s
810:	learn: 0.4044821	total: 38.7s	remaining: 32.8s
811:	learn: 0.4044709	total: 38.7s	remaining: 32.8s
812:	learn: 0.4044439	total: 38.8s	remaining: 32.8s
813:	learn: 0.4044228	total: 38.8s	remaining: 32.7s
814:	learn: 0.4043936	total: 38.9s	remaining: 32.7s
815:	learn: 0.4043690	total: 39.1s	remaining: 32.8s
816:	learn: 0.4043438	total: 39.4s	remaining: 33s
817:	learn: 0.4043301	total: 39.7s	remaining: 33.1s
818:	learn: 0.4042909	total: 40s	remaining: 33.3s
819:	learn: 0.4042463	total: 40.3s	remaining: 33.4s
820:	learn: 0.4042177	total: 40.6s	remaining: 33.6s
821:	learn: 0.4041655	total: 40.9s	remaining: 33.7s
822:	learn: 0.4041391	total: 41.2s	remaining: 33.9s
823:	learn: 0.4041316	total: 41.5s	remaining: 34s
824:	learn: 0.4040922	total: 41.8s	remaining: 34.2s
825:	learn: 0.4040775	

In [18]:
#print(clf_multilabel.best_score_)
mod = joblib.load("CAT_40.pkl")

In [19]:
y_pred = mod.predict(x_test)

In [20]:
y_pred

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]])

In [21]:
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))
#oooooooooooooooooooooooooooooooo ya

0.5337471312330482
0.4262707262996866
0.6876947014650964
0.20207980106250706


In [ ]:
#y_predtest_prob.to_pickle("./y_predtest_probcat.pkl")

In [ ]:
#y_predtrain_prob.to_pickle("./y_predtrain_probcat.pkl")